# Lights model with PBC2 dataset

In [1]:
# Library setup
%reset -f
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from lifelines import KaplanMeierFitter
from tick.plot import plot_point_process
from lights.simulation import SimuJointLongitudinalSurvival
from lights.base.utils import heatmap, annotate_heatmap, gompertz_pdf, \
                              gompertz_survival, visualize_vect_learning
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import LabelEncoder
from lifelines.utils import concordance_index as c_index_score
from IPython.display import Markdown, display
from scipy.stats import beta
from matplotlib import rc
rc('text', usetex=True)
from lights.inference import QNMCEM

def printmd(string):
    display(Markdown(string))

In [2]:
# dataset setup
pbc2 = pd.read_csv("pbc2.csv")
n_samples = pbc2["id"].drop_duplicates().shape[0]
long_features_list = ["serBilir", "albumin", "SGOT"]
n_long_features = len(long_features_list)

In [3]:
# data preprocessing
# survival data
survival_data = pbc2[["id", "years", "status2", "drug", "age", "sex"]].drop_duplicates()
X = survival_data[["drug", "age", "sex"]]
labelencoder = LabelEncoder()
X["drug"] = labelencoder.fit_transform(X["drug"])
X["sex"] = labelencoder.fit_transform(X["sex"])
X = X.values
T = survival_data[["years"]].values.flatten()
delta = survival_data[["status2"]].values.flatten()

# longitunal data
Y_ = pbc2[["id", "year", "serBilir", "albumin", "SGOT"]]
Y = pd.DataFrame(columns=long_features_list)
for i in range(n_samples):
    y_i = []
    for l in range(n_long_features):
        Y_il = Y_[["year", long_features_list[l]]][Y_["id"]==i+1]
        y_i += [pd.Series(Y_il[long_features_list[l]].values, index=Y_il["year"].values)]
    Y.loc[i] = y_i

In [4]:
## Choose parameters ##
tol = 1e-6            # tolerance for the convergence stopping criterion 
eta = 0.3             # parameter controlling the trade-off between l1 
                      # and l2 regularization in the elasticNet
gamma_chosen = '1se'  # way to select l_elasticNet_chosen: '1se' or 'min'
warm_start = True     # at each L-BGFS-B iteration, reset beta to 0 or take 
                      # the previous value 
grid_size = 30        # grid size for the cross validation procedure
metric = 'C-index'    # cross-validation metric: 'log_lik' or 'C-index'

# declare learner here
fixed_effect_time_order = 1
qnmcem = QNMCEM(fixed_effect_time_order=fixed_effect_time_order, max_iter=10, initialize=True, print_every=1,
               compute_obj=True)
# qnmcem.fit(X_train, Y_train, T_train, delta_train)
qnmcem.fit(X, Y, T, delta)

# Visualize learning
visualize_vect_learning(qnmcem, "obj")

## Cross-validation ##

## Run selected model with l_elasticNet_chosen ##

# run final fit here

Launching the solver QNMCEM...
Launching the solver MLMM...
Launching the solver ULMM...


/Users/acsi/Working/lights/.env/lib/python3.6/site-packages/statsmodels/regression/mixed_linear_model.py:2149: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Done solving using ULMM in 2.29e+00 seconds
 n_iter  |   obj    | rel_obj 
       0 |  19009.1 |      inf
       1 |  18977.3 | 1.68e-03
       2 |  19015.8 | 2.03e-03
       3 |    19006 | 5.14e-04
       4 |  19001.3 | 2.50e-04
       5 |  18999.7 | 8.27e-05
       6 |  19000.1 | 2.30e-05
       7 |  19001.9 | 9.50e-05
       8 |  19004.7 | 1.46e-04
       9 |  19008.2 | 1.85e-04
      10 |  19012.3 | 2.14e-04
Done solving using MLMM in 3.59e+00 seconds
 n_iter  |   obj    | rel_obj 
       0 |  64.0651 |      inf
       1 |  59.8687 | 6.55e-02
       2 |  59.3868 | 8.05e-03
       3 |  59.6989 | 5.26e-03
       4 |  59.1563 | 9.09e-03
       5 |  59.2809 | 2.11e-03
       6 |  58.8707 | 6.92e-03
       7 |  59.1763 | 5.19e-03
       8 |  59.3713 | 3.30e-03
       9 |  59.0903 | 4.73e-03
      10 |  58.8391 | 4.25e-03
Done solving using QNMCEM in 1.20e+03 seconds
